In [3]:
%reload_ext autoreload
%autoreload 2
from datasets import load_dataset


# ignore_verifications=Trueをつけないとエラーとなる
oscar_dataset = load_dataset('oscar', 'unshuffled_original_ja', 
                       split='train', 
                       ignore_verifications=True,
                       streaming=True)
#mc4の場合はこちら
mc4_dataset = load_dataset('mc4', 'ja',split='train', streaming=True)
dataset=oscar_dataset

/home/user/miniconda3/envs/crawl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/miniconda3/envs/crawl/lib/python3.12/site-packages/datasets/load.py:2516: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
/home/user/miniconda3/envs/crawl/lib/python3.12/site-packages/datasets/load.py:1454: FutureWarning: The repository for oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this

In [4]:
loc_records = []
cnt=0
for record in dataset:
    loc_records.append(record)
    cnt+=1
    if cnt>10**2:
        break

In [33]:
"""
sp_path = "../data/lm_sp/ja.sp.model"
arpa_path = "../data/lm_sp/ja.arpa.bin"
from src.cleaner.perplexity_checker import PerplexityChecker
perp_checker = PerplexityChecker(arpa_path,sp_path)
"""


'\nsp_path = "../data/lm_sp/ja.sp.model"\narpa_path = "../data/lm_sp/ja.arpa.bin"\nfrom src.cleaner.perplexity_checker import PerplexityChecker\nperp_checker = PerplexityChecker(arpa_path,sp_path)\n'

In [240]:
from src.cleaner.auto_cleaner import clean_text
for i in range(100):
    record=loc_records[i]
    original_text=record["text"]
    text=clean_text(original_text)
    if text!="":
        print(text)
    #print(original_text)

神社などへ一緒に同行して、様々な角度のショットで家族写真やお子様の写真を撮影致します!お好みに合わせて様々な写真を取ることができますので、その場でカメラマンへのリクエストも可能です!お子様の晴れ姿を、緊張していない自然な笑顔で残しませんか?
※七五三の対象者が1名のプランです。ご兄弟、ご友人様もご一緒になどと、主役が2名、3名と複数名いらっしゃる場合は、+5,000円(税抜)/1名の別途料金が発生いたします。
※撮影神社、宮社によっては撮影許可料(3,000円ほど)が別途発生いたす場所がございます。こちらは弊社撮影料金には含まれておりませんので、事前に撮影希望神社へお客様よりご確認、お支払いをお願いしております。ご確認の程よろしくお願いいたします。
◎正当な理由による書き込みの削除について:1レス¥5000円1スレ¥20000円の技術作業料が発生します。一回分だけの料金で当方管理下の全サイトで作業が実施されます。支払い方法はAmazonギフト券番号。連絡先は当サイトの登録emailへ。
騒音注意され腹立てた男、男性の顔殴り眼球破裂:社会:YOMIURI ONLINE(読売新聞)http://www.yomiuri.co.jp/national/news/20131106-OYT1T00269.htm怖い
名前使い回し効果も使いまわしの数値変更っていうのがなんかネタ切れ感感じますね・・・アンリミテッドで旧カードと合わせて6枚同名キャラが入ったファンデッキ作るのは面白そうですね旧カードと同カード扱いされて合計3枚制限がかかるかは心配ですが・・・
たまには、人を語らせて下さい。近野淳一さんと知り合ったのはここ数年の事、とある方(⬅すばらし方です)の紹介でお店に一緒に来たのがキッカケ。その後、交流があり昨年は3本もお店でライブをやって頂き、盛り上がったよな～と!更新しながらも思い出してしまいますが...ほんとに毎回近野淳一ワールドで、お客様の笑顔や喜ぶ姿が今でも目に焼き付いております。
さんび～なや(_ゝ`)本日は一段と冷え込みも激しく鼻っぱしらが氷りそう。さて、先月は秋田朝日放送さんの番組、人情食堂に出させて頂きました。正直、緊張しましたよ～テレビですよ、人生初めての体験!カメラを前にすると素顔がでてしまいそうで、普段からの店長としてのロールプレイ?が役にたった場面でもあ

'{"text": "本体:高さ27cm幅5cmちょこんと座ったパトリックの表情が、人(ヒトデ?)の良さを醸し出しています。ソフビ製のフィギュア部分は取り外すことが出来ます。中身はスポンジボブ形のカラフルなキャンディです。"}'